In [6]:
import numpy as np
from aisynphys.database import SynphysDatabase
from aisynphys.cell_class import CellClass, classify_cells, classify_pairs
from aisynphys.connectivity import measure_connectivity

In [7]:
db = SynphysDatabase(ro_host='sqlite:///', rw_host=None, db_name='/home/luke/public/synphys_current_small.sqlite')

Get a list of all cell pairs in the database from mouse matrix projects:

In [17]:
mouse_pairs = db.pair_query(project_name=["mouse V1 coarse matrix", "mouse V1 pre-production"]).all()
mouse_probed = [p for p in mouse_pairs if p.n_ex_test_spikes >= 10 or p.n_in_test_spikes >= 10]
mouse_connected = [p for p in mouse_pairs if p.has_synapse]

print("All mouse cell pairs:", len(mouse_pairs))
print("Mouse pairs that were probed for connectivity:", len(mouse_probed))
print("Mouse connected pairs:", len(mouse_connected))
print("Overall mouse connection probability:", len(mouse_connected) / len(mouse_probed))

All mouse cell pairs: 77084
Mouse pairs that were probed for connectivity: 22251
Mouse connected pairs: 1368
Overall mouse connection probability: 0.06148038290413914


In [18]:
human_pairs = db.pair_query(project_name=["human coarse matrix"]).all()
human_probed = [p for p in human_pairs if p.n_ex_test_spikes >= 10 or p.n_in_test_spikes >= 10]
human_connected = [p for p in human_pairs if p.has_synapse]

print("All human cell pairs:", len(human_pairs))
print("Human pairs that were probed for connectivity:", len(human_probed))
print("Human connected pairs:", len(human_connected))
print("Overall human connection probability:", len(human_connected) / len(human_probed))

All human cell pairs: 10410
Human pairs that were probed for connectivity: 3698
Human connected pairs: 363
Overall human connection probability: 0.0981611681990265


In [42]:
for species in ('human', 'mouse'):
    query = db.query(db.Cell, db.Morphology.cortical_layer)
    query = query.join(db.Morphology).join(db.Experiment).join(db.Slice)
    query = query.filter(db.Slice.species==species)
    cells = query.dataframe()
    n_cells_with_layer = sum(cells.cortical_layer.notnull())
    print("%d %s cells with cortical layer call" % (n_cells_with_layer, species))


835 human cells with cortical layer call
5251 mouse cells with cortical layer call


In [45]:
cells = db.query(db.Cell).all()
for species in ('human', 'mouse'):
    n_cells_with_layer = 0
    for cell in cells:
        if cell.experiment.slice.species != species:
            continue
        if cell.morphology is not None and cell.morphology.cortical_layer is not None:
            n_cells_with_layer += 1
            
    print("%d %s cells with cortical layer call" % (n_cells_with_layer, species))


835 human cells with cortical layer call
5251 mouse cells with cortical layer call


In [50]:
db.query(db.Metadata).all()[0].meta

{'db_version': '15',
 'creation_date': '2019-08-29',
 'origin': 'Allen Institute for Brain Science / Synaptic Physiology'}